In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Initialize

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
import copy
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import zscore
from sklearn.model_selection import train_test_split

has_mps = torch.backends.mps.is_built()
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

class EarlyStopping:
    def __init__(self, patience=5, min_delta=0, restore_best_weights=True):
        self.patience = patience
        self.min_delta = min_delta
        self.restore_best_weights = restore_best_weights
        self.best_model = None
        self.best_loss = None
        self.counter = 0
        self.status = ""

    def __call__(self, model, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
            self.best_model = copy.deepcopy(model.state_dict())
        elif self.best_loss - val_loss >= self.min_delta:
            self.best_model = copy.deepcopy(model.state_dict())
            self.best_loss = val_loss
            self.counter = 0
            self.status = f"Improvement found, counter reset to {self.counter}"
        else:
            self.counter += 1
            self.status = f"No improvement in the last {self.counter} epochs"
            if self.counter >= self.patience:
                self.status = f"Early stopping triggered after {self.counter} epochs."
                if self.restore_best_weights:
                    model.load_state_dict(self.best_model)
                return True
        return False

# Feature Engineering

In [ ]:
# Here 

In [ ]:
#onek kaj ache ekhane

# 🎯 One-hot encode target
x_columns = df.columns.drop("product").drop("id")
x = df[x_columns].values
dummies = pd.get_dummies(df["product"], dtype=int)
y = dummies.values
y_class = np.argmax(y, axis=1)  # Needed for StratifiedKFold

# ✅ Convert once to tensors
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
x_tensor = torch.tensor(x, dtype=torch.float32, device=device)
y_tensor = torch.tensor(y_class, dtype=torch.long, device=device)

In [ ]:
# 🧠 Model architecture
def build_model(input_dim, output_dim):
    return torch.compile(nn.Sequential(
        nn.Linear(input_dim, 64),
        nn.ReLU(),
        nn.Linear(64, 32),
        nn.ReLU(),
        nn.Linear(32, output_dim),
        nn.Softmax(dim=1),
    ), backend="aot_eager").to(device)

# 📊 Tracking
oos_y = []
oos_pred = []

kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
fold = 0

for train_idx, test_idx in kf.split(x_tensor.cpu(), y_tensor.cpu()):
    fold += 1
    print(f"\n📂 Fold #{fold}")

    # Split data
    x_train, x_test = x_tensor[train_idx], x_tensor[test_idx]
    y_train, y_test = y_tensor[train_idx], y_tensor[test_idx]

    train_ds = TensorDataset(x_train, y_train)
    train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)

    # Model setup
    model = build_model(x.shape[1], y.shape[1])
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    es = EarlyStopping(patience=10)
    EPOCHS = 100

    for epoch in range(1, EPOCHS + 1):
        model.train()
        for xb, yb in train_loader:
            optimizer.zero_grad()
            preds = model(xb)
            loss = criterion(preds, yb)
            loss.backward()
            optimizer.step()

        # Validation loss
        model.eval()
        with torch.no_grad():
            val_logits = model(x_test)
            val_loss = criterion(val_logits, y_test)

        print(f"Epoch {epoch} - Val Loss: {val_loss.item():.4f} ({es.status})")
        if es(model, val_loss.item()):
            print("⛔ Early stopping triggered.")
            break

    # 🔁 Load best weights
    model.load_state_dict(es.best_model_state)

    # 🎯 Final prediction
    model.eval()
    with torch.no_grad():
        final_probs = model(x_test)
        final_preds = torch.argmax(final_probs, dim=1)

    oos_y.append(y_test.cpu().numpy())
    oos_pred.append(final_preds.cpu().numpy())

    fold_acc = metrics.accuracy_score(y_test.cpu().numpy(), final_preds.cpu().numpy())
    print(f"✅ Fold Accuracy: {fold_acc:.4f}")

# 🧮 Final accuracy
oos_y = np.concatenate(oos_y)
oos_pred = np.concatenate(oos_pred)
final_acc = metrics.accuracy_score(oos_y, oos_pred)
print(f"\n🏁 Final Overall Accuracy: {final_acc:.4f}")

